# Importing libraries and data

In [ ]:
import csv
import math
import random
import numpy as np
import pandas as pd
import gdown
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
import plotly.express as px
import time
import random
from scipy.stats import beta

In [ ]:
file_id = '1EI8FuKQ5Hho_hfFADLWh4wJ2FFSy8vZS'
url = f'https://drive.google.com/uc?id={file_id}'
output = 'purchases_final_PLEASE_TY.csv'
gdown.download(url, output, quiet=False)

file_id = '1PZCA2ytJtFpkPRRmvyNTRzJO26aiJ9Ot'
url = f'https://drive.google.com/uc?id={file_id}'
output = 'q2_customers.csv'
gdown.download(url, output, quiet=False)


Downloading...
From (original): https://drive.google.com/uc?id=1EI8FuKQ5Hho_hfFADLWh4wJ2FFSy8vZS
From (redirected): https://drive.google.com/uc?id=1EI8FuKQ5Hho_hfFADLWh4wJ2FFSy8vZS&confirm=t&uuid=c6035e4d-1db4-4f69-a7bc-052056449550
To: /content/purchases_final_PLEASE_TY.csv
100%|██████████| 272M/272M [00:01<00:00, 155MB/s]
Downloading...
From: https://drive.google.com/uc?id=1YQOfdb4ICBN3I-SAfrYM0714NSHC-jY2
To: /content/q2_customers.csv
100%|██████████| 915k/915k [00:00<00:00, 35.1MB/s]


'q2_customers.csv'

In [ ]:
customers = pd.read_csv('q2_customers.csv')
purchases = pd.read_csv('purchases_final_PLEASE_TY.csv')

In [ ]:
customers.head()

,Survey ResponseID,Prime,Email CTC,Display Ad CTC,Video Ad CTC,Search Engine Ad CTC,Social Media Ad CTC,Average CTC,Express Shipping,Cart Abandonment Rate,Check Review,Refund rate,Time in Cart,Time on Page,RFM,Pct Discount,Keep Discount,Category
0,R_03aEbghUILs9NxD,0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.428571,0.733693,0,0.243994,7.039021,0.469268,111,0.562500,1,NaN
1,R_06RZP9pS7kONINr,1,0.033368,0.032189,0.000000,0.000000,0.0,0.0,1.000000,0.709685,1,0.170417,45.151821,3.010121,323,0.458128,1,NaN
2,R_085qq7w0pkhowox,0,0.000000,0.000000,0.006171,0.000000,0.0,0.0,0.475410,0.719513,1,0.306721,19.751185,1.316746,322,0.454861,1,NaN
3,R_08uYA7fb4unHGkF,1,0.000000,0.000000,0.000000,0.000000,0.0,0.0,1.000000,0.645122,0,0.118960,13.694138,0.912943,212,0.382022,1,NaN
4,R_0BrMxYYMHVMOBEJ,0,0.000000,0.042679,0.000000,0.021435,0.0,0.0,0.500000,0.832793,0,0.343723,2.544373,0.169625,211,0.380952,1,NaN


In [ ]:
purchases.head()

,Order Date,Purchase Price Per Unit,Quantity,Shipping Address State,Title,ASIN/ISBN (Product Code),Survey ResponseID,Category,Prime,Discounts,Express Shipping
0,2018-12-04,7.98,1,NJ,SanDisk Ultra 16GB Class 10 SDHC UHS-I Memory ...,B0143RTB1E,R_01vNIayewjIIKMF,Electronics,1,0.00,1
1,2018-12-22,13.99,1,NJ,Betron BS10 Earphones Wired Headphones in Ear ...,B01MA1MJ6H,R_01vNIayewjIIKMF,Electronics,1,0.00,1
2,2018-12-24,8.99,1,NJ,NaN,B078JZTFN3,R_01vNIayewjIIKMF,NaN,1,0.00,1
3,2018-12-25,10.45,1,NJ,Perfecto Stainless Steel Shaving Bowl. Durable...,B06XWF9HML,R_01vNIayewjIIKMF,Kitchen & Dining,1,0.05,1
4,2018-12-25,10.00,1,NJ,Proraso Shaving Cream for Men,B00837ZOI0,R_01vNIayewjIIKMF,Beauty & Personal Care,1,0.05,1


# Customer Segmentation

## Saving time/urgency (for non-Prime members)

In [ ]:
urgency = customers[customers['Express Shipping'] >= 0.5]
urgency = urgency[urgency['Time in Cart'] <= 20]
urgency = urgency[urgency['Prime'] == 0]
len(urgency)

364

In [ ]:
len(urgency) / len(customers[customers['Prime'] == 0])


0.21125943122460825

about 21% of non-prime users are more urgent (will pay for expedited shipping more often).

In [ ]:
urgency['Cat_Urgency'] = 1
urgency['Cat_Urgency'] = urgency['Cat_Urgency'].astype(int)
urgency = urgency[['Survey ResponseID', 'Cat_Urgency']]
customers = pd.merge(customers, urgency, on='Survey ResponseID', how='left')
customers['Cat_Urgency'] = customers['Cat_Urgency'].fillna(0).astype(int)
customers.head()

MergeError: Passing 'suffixes' which cause duplicate columns {'Cat_Urgency_x'} is not allowed.

## Impulse buyer

In [ ]:
impulse = customers[customers['Time in Cart'] <= 20]
impulse = impulse[impulse['Time on Page'] <= 2.5]
impulse = impulse[impulse['Check Review'] == 0 ]
impulse = impulse[impulse['Refund rate'] > 0.05]
len(impulse)

In [ ]:
len(impulse) / len(customers)

about 36% of customers are impulse buyers.

In [ ]:
impulse['Cat_Impulse'] = 1
impulse['Cat_Impulse'] = impulse['Cat_Impulse'].astype(int)
impulse = impulse[['Survey ResponseID', 'Cat_Impulse']]
customers = pd.merge(customers, impulse, on='Survey ResponseID', how='left')
customers['Cat_Impulse'] = customers['Cat_Impulse'].fillna(0).astype(int)
customers.head()

## Indecisive/cautious

In [ ]:
indecisive = customers[customers['Cart Abandonment Rate'] >= 0.75]
indecisive = indecisive[indecisive['Check Review'] == 1]
indecisive = indecisive[indecisive['Time on Page'] >= 7]
indecisive = indecisive[indecisive['Time in Cart'] >= 30]
len(indecisive)

In [ ]:
len(indecisive) / len(customers)

about 13% of customers are indecisive buyers

In [ ]:
indecisive['Cat_Indecisive'] = 1
indecisive['Cat_Indecisive'] = indecisive['Cat_Indecisive'].astype(int)
indecisive = indecisive[['Survey ResponseID', 'Cat_Indecisive']]
customers = pd.merge(customers, indecisive, on='Survey ResponseID', how='left')
customers['Cat_Indecisive'] = customers['Cat_Indecisive'].fillna(0).astype(int)
customers.head()

## Category

In [ ]:
cat = customers[customers['Category'].notnull()]
len(cat)

In [ ]:
len(indecisive) / len(customers)

about 13% of customers have a frequently purchased category that they stick to when ordering

In [ ]:
cat['Cat_Category'] = 1
cat['Cat_Category'] = cat['Cat_Category'].astype(int)
cat = cat[['Survey ResponseID', 'Cat_Category']]
customers = pd.merge(customers, cat, on='Survey ResponseID', how='left')
customers['Cat_Category'] = customers['Cat_Category'].fillna(0).astype(int)
customers.head()

## Price sensitive

In [ ]:
discount = customers[customers['Keep Discount'] == 1]
discount = customers[customers['Pct Discount'] >= 0.4]
len(discount)

In [ ]:
len(discount) / len(customers)

In [ ]:
discount['Cat_Discount'] = 1
discount['Cat_Discount'] = discount['Cat_Discount'].astype(int)
discount = discount[['Survey ResponseID', 'Cat_Discount']]
customers = pd.merge(customers, discount, on='Survey ResponseID', how='left')
customers['Cat_Discount'] = customers['Cat_Discount'].fillna(0).astype(int)
customers.head()

In [ ]:
customers = customers.drop(columns = ['Keep Discount'])

## Marketing Engagement

In [ ]:
cols_to_check = ['Email CTC', 'Display Ad CTC', 'Video Ad CTC', 'Search Engine Ad CTC', 'Social Media Ad CTC']
filtered_cust = customers[customers[cols_to_check].gt(0).any(axis=1)]
len(filtered_cust)

In [ ]:
len(filtered_cust) / len(customers)

about 88% of customers were engaged by any one of the marketing ads

In [ ]:
filtered_cust['Cat_Engagement'] = 1
filtered_cust['Cat_Engagement'] = filtered_cust['Cat_Engagement'].astype(int)
filtered_cust = filtered_cust[['Survey ResponseID', 'Cat_Engagement']]
customers = pd.merge(customers, filtered_cust, on='Survey ResponseID', how='left')
customers['Cat_Engagement'] = customers['Cat_Engagement'].fillna(0).astype(int)
customers.head()

## High Value Customers

In [ ]:
hvc = customers.copy()
hvc[['R', 'F', 'M']] = hvc['RFM'].astype(str).apply(lambda x: pd.Series(list(x)))
hvc[['R', 'F', 'M']] = hvc[['R', 'F', 'M']].astype(int)
hvc.head()

In [ ]:
hvc_f = hvc[hvc['F'] == 3]
len(hvc_f) / len(customers)

about 25% of customers are high value (frequent) customers

In [ ]:
hvc_f['Cat_High_Value_F'] = 1
hvc_f['Cat_High_Value_F'] = hvc_f['Cat_High_Value_F'].astype(int)
hvc_f = hvc_f[['Survey ResponseID', 'Cat_High_Value_F']]
customers = pd.merge(customers, hvc_f, on='Survey ResponseID', how='left')
customers['Cat_High_Value_F'] = customers['Cat_High_Value_F'].fillna(0).astype(int)
customers.head()

In [ ]:
hvc_m = hvc[hvc['M'] == 3]
len(hvc_m) / len(customers)

about 25% of customers are high value (high spending) customers

In [ ]:
hvc_m['Cat_High_Value_M'] = 1
hvc_m['Cat_High_Value_M'] = hvc_m['Cat_High_Value_M'].astype(int)
hvc_m = hvc_m[['Survey ResponseID', 'Cat_High_Value_M']]
customers = pd.merge(customers, hvc_m, on='Survey ResponseID', how='left')
customers['Cat_High_Value_M'] = customers['Cat_High_Value_M'].fillna(0).astype(int)
customers.head()

In [ ]:
customers.info()

## Analysis

In [ ]:
customers.head()

In [ ]:
customers.info()

In [ ]:
label_count = customers.iloc[:, 17:25].sum(axis=0)
print(label_count)

In [ ]:
label_percentage = (label_count / len(customers)) * 100
print(label_percentage)

In [ ]:
impulse_engaged = customers[(customers['Cat_Engagement'] == 1) & (customers['Cat_Impulse'] == 1)].shape[0]
impulse_engaged / 1779 * 100

In [ ]:
discount_indecisive = customers[(customers['Cat_Indecisive'] == 1) & (customers['Cat_Discount'] == 1)].shape[0]
discount_indecisive / 3460 * 100

# RFM

In [ ]:
def rfm(purchases,customers):
  customers['RFM'] = customers['RFM'].astype(str)
  customers['Churn Risk'] = customers['RFM'].str[0].apply(lambda x: "Active" if x == "3" else ("At risk" if x == "2" else "Churned"))
  return(customers)
print(customers.columns)

Index(['Survey ResponseID', 'Prime', 'Email CTC', 'Display Ad CTC',
       'Video Ad CTC', 'Search Engine Ad CTC', 'Social Media Ad CTC',
       'Average CTC', 'Express Shipping', 'Cart Abandonment Rate',
       'Check Review', 'Refund rate', 'Time in Cart', 'Time on Page', 'RFM',
       'Pct Discount', 'Category', 'Cat_Urgency', 'Cat_Impulse',
       'Cat_Indecisive', 'Cat_Category', 'Cat_Discount', 'Cat_Engagement',
       'Cat_High_Value_F', 'Cat_High_Value_M', 'Churn Risk'],
      dtype='object')


In [ ]:
#urgency
customers_urgency = customers[customers['Cat_Urgency'] == 1]
trans_urgency = pd.merge(customers_urgency, purchases, on='Survey ResponseID', how='left')
rfm_urgency = rfm(trans_urgency, customers_urgency)

#impulse
customers_impulse = customers[customers['Cat_Impulse'] == 1]
trans_impulse = pd.merge(customers_impulse, purchases, on='Survey ResponseID', how='left')
rfm_impulse = rfm(trans_impulse, customers_impulse)

#indecisive
customers_indecisive = customers[customers['Cat_Indecisive'] == 1]
trans_indecisive = pd.merge(customers_indecisive, purchases, on='Survey ResponseID', how='left')
rfm_indecisive = rfm(trans_indecisive, customers_indecisive)

#category
customers_category = customers[customers["Cat_Category"] == 1]
trans_category = pd.merge(customers_category, purchases, on='Survey ResponseID', how='left')
rfm_category = rfm(trans_category, customers_category)

#engagement
customers_engagement = customers[customers["Cat_Engagement"] == 1]
trans_engagement = pd.merge(customers_engagement, purchases, on='Survey ResponseID', how='left')
rfm_engagement = rfm(trans_engagement, customers_engagement)

#high value F
customers_highvalf = customers[customers['Cat_High_Value_F'] == 1]
trans_highvalf = pd.merge(customers_highvalf, purchases, on='Survey ResponseID', how='left')
rfm_highvalf = rfm(trans_highvalf, customers_highvalf)

#high value M
customers_highvalm = customers[customers['Cat_High_Value_M'] == 1]
trans_highvalm = pd.merge(customers_highvalm, purchases, on='Survey ResponseID', how='left')
rfm_highvalm = rfm(trans_highvalm, customers_highvalm)

#discount
customers_discount = customers[customers["Cat_Discount"] == 1]
trans_discount = pd.merge(customers_discount, purchases, on='Survey ResponseID', how='left')
rfm_discount = rfm(trans_discount, customers_discount)

churn_segments_data = {
    'Segment': [],
    'Active': [],
    'At risk': [],
    'Churned': [],
    'Size': []
}

churn_segments = pd.DataFrame(churn_segments_data)

churn_segments.loc[len(churn_segments)] = ["Urgency", rfm_urgency[rfm_urgency['Churn Risk'] == "Active"].shape[0]/rfm_urgency.shape[0],
                                           rfm_urgency[rfm_urgency['Churn Risk'] == "At risk"].shape[0]/rfm_urgency.shape[0],
                                           rfm_urgency[rfm_urgency['Churn Risk'] == "Churned"].shape[0]/rfm_urgency.shape[0],
                                           rfm_urgency.shape[0]/customers.shape[0]]
churn_segments.loc[len(churn_segments)] = ["Impulse", rfm_impulse[rfm_impulse['Churn Risk'] == "Active"].shape[0]/rfm_impulse.shape[0],
                                           rfm_impulse[rfm_impulse['Churn Risk'] == "At risk"].shape[0]/rfm_impulse.shape[0],
                                           rfm_impulse[rfm_impulse['Churn Risk'] == "Churned"].shape[0]/rfm_impulse.shape[0],
                                           rfm_impulse.shape[0]/customers.shape[0]]
churn_segments.loc[len(churn_segments)] = ["Indecisive", rfm_indecisive[rfm_indecisive['Churn Risk'] == "Active"].shape[0]/rfm_indecisive.shape[0],
                                           rfm_indecisive[rfm_indecisive['Churn Risk'] == "At risk"].shape[0]/rfm_indecisive.shape[0],
                                           rfm_indecisive[rfm_indecisive['Churn Risk'] == "Churned"].shape[0]/rfm_indecisive.shape[0],
                                           rfm_indecisive.shape[0]/customers.shape[0]]
churn_segments.loc[len(churn_segments)] = ["Category", rfm_category[rfm_category['Churn Risk'] == "Active"].shape[0]/rfm_category.shape[0],
                                           rfm_category[rfm_category['Churn Risk'] == "At risk"].shape[0]/rfm_category.shape[0],
                                           rfm_category[rfm_category['Churn Risk'] == "Churned"].shape[0]/rfm_category.shape[0],
                                           rfm_category.shape[0]/customers.shape[0]]
churn_segments.loc[len(churn_segments)] = ["Engagement", rfm_engagement[rfm_engagement['Churn Risk'] == "Active"].shape[0]/rfm_engagement.shape[0],
                                           rfm_engagement[rfm_engagement['Churn Risk'] == "At risk"].shape[0]/rfm_engagement.shape[0],
                                           rfm_engagement[rfm_engagement['Churn Risk'] == "Churned"].shape[0]/rfm_engagement.shape[0],
                                           rfm_engagement.shape[0]/customers.shape[0]]
churn_segments.loc[len(churn_segments)] = ["High Value Frequency", rfm_highvalf[rfm_highvalf['Churn Risk'] == "Active"].shape[0]/rfm_highvalf.shape[0],
                                           rfm_highvalf[rfm_highvalf['Churn Risk'] == "At risk"].shape[0]/rfm_highvalf.shape[0],
                                           rfm_highvalf[rfm_highvalf['Churn Risk'] == "Churned"].shape[0]/rfm_highvalf.shape[0],
                                           rfm_highvalf.shape[0]/customers.shape[0]]
churn_segments.loc[len(churn_segments)] = ["High Value Monetary", rfm_highvalm[rfm_highvalm['Churn Risk'] == "Active"].shape[0]/rfm_highvalm.shape[0],
                                           rfm_highvalm[rfm_highvalm['Churn Risk'] == "At risk"].shape[0]/rfm_highvalm.shape[0],
                                           rfm_highvalm[rfm_highvalm['Churn Risk'] == "Churned"].shape[0]/rfm_highvalm.shape[0],
                                           rfm_highvalm.shape[0]/customers.shape[0]]
churn_segments.loc[len(churn_segments)] = ["Discount", rfm_discount[rfm_discount['Churn Risk'] == "Active"].shape[0]/rfm_discount.shape[0],
                                           rfm_discount[rfm_discount['Churn Risk'] == "At risk"].shape[0]/rfm_discount.shape[0],
                                           rfm_discount[rfm_discount['Churn Risk'] == "Churned"].shape[0]/rfm_discount.shape[0],
                                           rfm_discount.shape[0]/customers.shape[0]]

churn_segments.head(10)

<ipython-input-50-70e0616cf76a>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  customers['RFM'] = customers['RFM'].astype(str)
<ipython-input-50-70e0616cf76a>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  customers['Churn Risk'] = customers['RFM'].str[0].apply(lambda x: "Active" if x == "3" else ("At risk" if x == "2" else "Churned"))
<ipython-input-50-70e0616cf76a>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = v

,Segment,Active,At risk,Churned,Size
0,Urgency,0.390110,0.321429,0.288462,0.073939
1,Impulse,0.654862,0.211355,0.133783,0.361365
2,Indecisive,0.553772,0.216693,0.229535,0.126549
3,Category,0.465517,0.290640,0.243842,0.082470
4,Engagement,0.602986,0.223933,0.173081,0.870810
5,High Value Frequency,0.893841,0.067261,0.038898,0.250660
6,High Value Monetary,0.891145,0.064175,0.044679,0.250051
7,Discount,0.642775,0.201445,0.155780,0.702823


In [ ]:
rfm_total = rfm(purchases, customers)

a = [rfm_total[rfm_total['Churn Risk'] == "Active"].shape[0]/rfm_total.shape[0], rfm_total[rfm_total['Churn Risk'] == "At risk"].shape[0]/rfm_total.shape[0], rfm_total[rfm_total['Churn Risk'] == "Churned"].shape[0]/rfm_total.shape[0]]
print(a)

[0.6049157018078407, 0.2208003250050782, 0.17428397318708105]


#Customer Lifetime Value

In [ ]:
print(customers.columns)

Index(['Survey ResponseID', 'Prime', 'Email CTC', 'Display Ad CTC',
       'Video Ad CTC', 'Search Engine Ad CTC', 'Social Media Ad CTC',
       'Average CTC', 'Express Shipping', 'Cart Abandonment Rate',
       'Check Review', 'Refund rate', 'Time in Cart', 'Time on Page', 'RFM',
       'Pct Discount', 'Keep Discount', 'Category'],
      dtype='object')
